# 00 — Setup & Sanity Checks

## Objective
This notebook verifies that the local development environment is ready for the Kigali EMS project.

We confirm:
- Python + virtual environment are active
- Core package imports work (data, geospatial, SUMO bindings)
- Environment variables (e.g., `SUMO_HOME`) are visible
- SUMO binaries (e.g., `sumo`, `sumo-gui`) are discoverable

**Important:** This notebook does **not** run a SUMO simulation yet. It only performs lightweight checks.

## 1.0 Assumptions & Scope

### In scope
- Print Python and platform details
- Import key dependencies and print versions
- Check `SUMO_HOME` and whether common SUMO commands are available
- Provide clear next steps if something is missing

### Out of scope (for now)
- Downloading OSM data
- Converting networks with `netconvert`
- Running any SUMO scenario
- Training RL agents

In [2]:
import os
import sys
import platform
from pathlib import Path
import shutil
import subprocess


def print_kv(name: str, value) -> None:
    """Print a labelled key/value pair for readable debugging."""
    print(f"{name}: {value}")


print("## Environment Diagnostics")
print_kv("Python executable", sys.executable)
print_kv("Python version", sys.version.replace("\n", " "))
print_kv("Platform", platform.platform())
print_kv("Working directory", Path.cwd())

# Quick venv hint (not perfect, but helpful)
venv_path = os.environ.get("VIRTUAL_ENV")
print_kv("VIRTUAL_ENV", venv_path if venv_path else "(not set)")

## Environment Diagnostics
Python executable: /Users/testsolutions/Documents/Academics/mission-capstone/marl-in-ems/.venv/bin/python
Python version: 3.12.5 (main, Jul 10 2025, 21:14:29) [Clang 16.0.0 (clang-1600.0.26.4)]
Platform: macOS-15.1.1-arm64-arm-64bit
Working directory: /Users/testsolutions/Documents/Academics/mission-capstone/marl-in-ems/notebooks
VIRTUAL_ENV: /Users/testsolutions/Documents/Academics/mission-capstone/marl-in-ems/.venv


## 2.0 Dependencies & Imports

In this section we:
- Import core libraries used throughout the project
- Print versions to help with debugging and reproducibility

If an import fails, we will see it early (before we build anything heavier).

In [3]:
def safe_import(module_name: str):
    """Import a module and return it; print a clear error if it fails."""
    try:
        module = __import__(module_name)
        return module
    except Exception as exc:
        print(f"[FAIL] Could not import '{module_name}': {exc}")
        return None


print("## Import Checks")

np = safe_import("numpy")
pd = safe_import("pandas")
matplotlib = safe_import("matplotlib")
ox = safe_import("osmnx")
gpd = safe_import("geopandas")

sumolib = safe_import("sumolib")
traci = safe_import("traci")

print("\n## Versions")
if np:
    print("numpy:", np.__version__)
if pd:
    print("pandas:", pd.__version__)
if matplotlib:
    print("matplotlib:", matplotlib.__version__)
if ox:
    print("osmnx:", ox.__version__)
if gpd:
    print("geopandas:", gpd.__version__)

# SUMO python libs do not always expose __version__
if sumolib:
    print("sumolib:", getattr(sumolib, "__version__", "unknown"))
if traci:
    print("traci: imported (version may be bundled with SUMO)")

## Import Checks

## Versions
numpy: 2.4.2
pandas: 3.0.0
matplotlib: 3.10.8
osmnx: 2.0.7
geopandas: 1.1.2
sumolib: 1.26.0
traci: imported (version may be bundled with SUMO)


## 3.0 Configuration & Paths

This project uses environment variables to locate external tools (especially SUMO).

We check:
- Whether `SUMO_HOME` is set
- Whether it points to a real directory
- Whether it contains typical SUMO subfolders (a light sanity check)

In [4]:
print("## SUMO_HOME Check")

sumo_home = os.environ.get("SUMO_HOME")
print_kv("SUMO_HOME", sumo_home if sumo_home else "(not set)")

if not sumo_home:
    print("\n[WARN] SUMO_HOME is not set yet.")
    print("       Later, create a .env file from .env.example and set SUMO_HOME for your machine.")
else:
    sumo_path = Path(sumo_home)
    print_kv("SUMO_HOME exists", sumo_path.exists())
    print_kv("SUMO_HOME is dir", sumo_path.is_dir())

    # Very light structure check (not strict)
    expected_candidates = ["tools", "bin"]
    found = {name: (sumo_path / name).exists() for name in expected_candidates}
    print("SUMO_HOME contains:")
    for k, v in found.items():
        print(f"  - {k}: {v}")

## SUMO_HOME Check
SUMO_HOME: /opt/homebrew/opt/sumo/share/sumo
SUMO_HOME exists: False
SUMO_HOME is dir: False
SUMO_HOME contains:
  - tools: False
  - bin: False


## 4.0 SUMO Installation Checks

Even if Python can import `traci`, we still need the SUMO installation:
- `sumo` binary should be available
- SUMO tools folder should exist
- `SUMO_HOME` should point to the SUMO share directory (used by tools and examples)

We will try a few checks and suggest a correct `SUMO_HOME` for Homebrew installs.

In [5]:
print("## SUMO Binary Checks")

sumo_bin = shutil.which("sumo")
sumo_gui_bin = shutil.which("sumo-gui")
print_kv("which sumo", sumo_bin if sumo_bin else "(not found)")
print_kv("which sumo-gui", sumo_gui_bin if sumo_gui_bin else "(not found)")

if sumo_bin:
    try:
        out = subprocess.check_output([sumo_bin, "--version"], text=True).strip()
        print("\nSUMO version output:")
        print(out)
    except Exception as exc:
        print(f"\n[WARN] Could not run 'sumo --version': {exc}")

print("\n## Homebrew SUMO prefix (if available)")
brew_bin = shutil.which("brew")
print_kv("which brew", brew_bin if brew_bin else "(not found)")

if brew_bin:
    try:
        prefix = subprocess.check_output(["brew", "--prefix", "sumo"], text=True).strip()
        print_kv("brew --prefix sumo", prefix)

        # Suggested SUMO_HOME for Homebrew
        candidate = Path(prefix) / "share" / "sumo"
        print_kv("candidate SUMO_HOME", str(candidate))
        print_kv("candidate exists", candidate.exists())

        if candidate.exists():
            print("\n[OK] Suggested SUMO_HOME exists. You can set:")
            print(f"SUMO_HOME={candidate}")
    except Exception as exc:
        print(f"[WARN] Could not query Homebrew prefix for SUMO: {exc}")

## SUMO Binary Checks
which sumo: (not found)
which sumo-gui: (not found)

## Homebrew SUMO prefix (if available)
which brew: /opt/homebrew/bin/brew


Error: No available formula with the name "sumo". Did you mean somo?


[WARN] Could not query Homebrew prefix for SUMO: Command '['brew', '--prefix', 'sumo']' returned non-zero exit status 1.


## 5.0 Summary & Next Steps

### Status
- ✅ Python + venv working
- ✅ Core Python dependencies imported successfully
- ⚠️ SUMO binaries (`sumo`, `sumo-gui`) not found
- ⚠️ `SUMO_HOME` currently points to a non-existent path

### Next steps (recommended)
1. Install SUMO (macOS)
2. Set `SUMO_HOME` to the correct SUMO share directory
3. Re-run this notebook to confirm `sumo --version` works

After SUMO is installed, we proceed to:
- `01_osm_to_sumo.ipynb` (download Kigali OSM network and convert to SUMO network)

In [6]:
print("## Final Checklist")

checks = {
    "venv_active": bool(os.environ.get("VIRTUAL_ENV")),
    "imports_ok": all([np, pd, matplotlib, ox, gpd, sumolib, traci]),
    "sumo_binary_found": bool(sumo_bin),
    "sumo_home_exists": bool(sumo_home) and Path(sumo_home).exists(),
}

for name, ok in checks.items():
    status = "PASS" if ok else "FAIL"
    print(f"{status:4} - {name}")

if not checks["sumo_binary_found"]:
    print("\nNext action: install SUMO and re-run this notebook.")

## Final Checklist
PASS - venv_active
PASS - imports_ok
FAIL - sumo_binary_found
FAIL - sumo_home_exists

Next action: install SUMO and re-run this notebook.
